In [9]:
include("all_code/code_needed.jl")
# example:
r = 3; #three communities for now
n0 = [100,100,100] #number of nodes in each community
p = 0.9 #whithin community
q = 0.1 #between communities
A,communities = generate_SBM_data(r,n0,p,q)
M = create_subnetworks(A,r,n0)
# M has 6 matrices as follows:
# 1 x x
# 2 3 x
# 4 5 6
# M[i] = the block number in the above picture

M0 = A
M1 = vcat(hcat(spzeros(M[1]),M[2]',M[4]'),hcat(M[2],spzeros(M[3]),M[5]'),hcat(M[4],M[5],spzeros(M[6])))
# M1 has the same matrix with zero matrices on the diagonal
;

In [10]:
# setup first:
#=============#
# dropping random edges between two networks (A-C [M4]) and (B-C [M5])
tao = 0.8 #percent to keep in training
M5train,M5test = split_train_test(M[5],tao);
M4train,M4test = split_train_test(M[4],tao);

# # Mtrain = [M[1] M[2]' M4train';
# #           M[2] M[3]  M[5]';
# #           M4train M[5] M[6]]
# # Mtrain = [M[1] M[2]' M[4]';
# #           M[2] M[3]  M5train';
# #           M[4] M5train M[6]]

Mtrain = [M[1] M[2]' M4train';
          M[2] M[3]  M5train';
          M4train M5train M[6]]

myalpha = 0.8
ii,jj,vv = findnz(M[2])

# X1 = M[5][:,ii]
# X2 = M4test[:,jj]
X1 = M5test[:,ii] # ==> X1[:,1] is all the "k" connections to nodes ii
X2 = M4test[:,jj] # ==> X2[:,1] is all the "k" connections to nodes jj

Xtest = X1.*X2;

# idstokeep = 1:size(Xtest,2)
# idstokeep = findall(sum(Xtest,dims=1)[:].!=0)[1:100]
idstokeep = 1:100

Xtest = Xtest[:,idstokeep]
ii = ii[idstokeep]
jj = jj[idstokeep]
# Xtest[:,1] is the "k" connections with the pair ii[1] and jj[1]

exps = length(ii)
seed1 = jj;
seed2 = ii;
# (want to seed on all edges connecting ii and jj)

#initialize a few things
myalpha = 0.8
Mc = copy(Mtrain) #copy(M0)

all_aucs_new = zeros(exps,4)
Xresult = zeros(size(Xtest));
Xresult1 = zeros(size(Xtest));
Xresult2 = zeros(size(Xtest));
Xresult3 = zeros(size(Xtest));
Xresult4 = zeros(size(Xtest));
;


In [12]:
for ei = 1:exps
    v = spzeros(size(Mc,2))
    vi = copy(v)
    vj = copy(v)
    rw = [seed1[ei],seed2[ei]]

    i = rw[1]
    j = rw[2]
    jm = n0[1] + j
    
    v[i] = 0.5
    v[jm] = 0.5
    vi[i] = 1
    vj[jm] = 1
    
    xrefreal = Xtest[:,ei]
    
    xsol = seeded_pagerank(Mc,myalpha,v)
    xsol1 = seeded_pagerank(Mc,myalpha,vi)
    xsol2 = seeded_pagerank(Mc,myalpha,vj)
    
    xsol3 = xsol1.*xsol2 
    xsol4 = xsol1.+xsol2
    
    xk = xsol[n0[1]+n0[2]+1:end]#DS
    xk1 = xsol1[n0[1]+n0[2]+1:end]
    xk2 = xsol2[n0[1]+n0[2]+1:end]#SS
    xk3 = xsol3[n0[1]+n0[2]+1:end]#SS-A
    xk4 = xsol4[n0[1]+n0[2]+1:end]#SS-O
    
    Xresult[:,ei] = xk
    Xresult1[:,ei] = xk1
    Xresult2[:,ei] = xk2
    Xresult3[:,ei] = xk3
    Xresult4[:,ei] = xk4
end

In [13]:
v = 1:size(Xtest,2)
tpr,fpr,auc = calc_AUC_new(Xtest[:,v],Xresult[:,v]); @show auc
tpr,fpr,auc = calc_AUC_new(Xtest[:,v],Xresult1[:,v]); @show auc
tpr,fpr,auc = calc_AUC_new(Xtest[:,v],Xresult2[:,v]); @show auc
tpr,fpr,auc = calc_AUC_new(Xtest[:,v],Xresult3[:,v]); @show auc
tpr,fpr,auc = calc_AUC_new(Xtest[:,v],Xresult4[:,v]); @show auc
;

auc = 0.47875937968984494
auc = 0.5018609304652326
auc = 0.5186793396698349
auc = 0.47963981990995497
auc = 0.47875937968984494


In [29]:
# collapsing the networks
# split the data first.
# predicting links related to (A,B)
tao = 0.8
myalpha = 0.8
i1,j1,TR = find_edges_of_tris(M);
TR = spones(TR)
nrows,ncols = size(TR)
C = vcat(
        hcat(spzeros(Int64,nrows,nrows),TR),
        hcat(TR',spzeros(Int64,ncols,ncols))
        )


Mtrain,Mtest = split_train_test(C,tao);
Ctest = Mtest[nrows+1:end,1:nrows]


seedon = 1:nrows

X = zeros(Float64,ncols,length(seedon));

for i = 1:length(seedon)
  X[:,i] = seeded_pagerank(Mtrain,myalpha,seedon[i])[nrows+1:end]
end
tpr2,fpr2,auc2 = calc_AUC_new(Ctest,X); @show auc2


auc2 = 0.40254277786687065


0.40254277786687065